# Linking PPO to vanilla policy gradient

- There's a connection between PPO and multi-step vanilla policy gradient that lends itself to a relatively easy understanding of what PPO is and why it's a good idea
- Unfortunately, the [PPO paper](https://arxiv.org/pdf/1707.06347.pdf) focuses on PPO as a more practical alternative to the highly complex TRPO, without connecting back to simpler policy gradient methods. And then the justifications of the methodology are hidden in the dense theoretical discussion of the [TRPO Paper](https://arxiv.org/pdf/1502.05477.pdf) which makes understanding the nature of PPO quite hard, unless you're an expert in RL and can comfortably navigate TRPO.
- TRPO also more focuses on monotonic improvement guarantees, rather than the importance sampling interpretation here which provides a much more understandable lead in to PPO. PPO has none of the theoretical monotonic guarantees, and performs SGA in its inner loop unlike TRPO.
- What I want to do here is some very basic mathematical analysis of vanilla policy gradient, and show that the PPO objective very naturally arises when we try to make VPG more data efficient by taking multiple steps on the same data. This explanation is accessible to anyone who's done a first course in probability, unlike the TRPO paper.
- There are 4 sections of this explanation

1. A basic intro to vanilla policy gradient
2. A definition of importance sampling
3. The problems that arise when we take multiple VPG steps
4. How PPO corrects these problems

## 1. Vanilla Policy Gradient

Suppose we want to estimate a gradient of an expectation of a function $R$, under some distribution $P$ parameterized by $\theta$:

$$\begin{align*}\nabla_{\theta}\mathbb{E}_{X \sim P_{\theta}}[R(X)]

&=\nabla_{\theta}\int_X p(x;\theta) R(x)  \: \mathrm{d}x\\

&=\int_X \nabla_{\theta} p(x;\theta) R(x)  \: \mathrm{d}x \hspace{10 mm}&(1) \text{ (Leibniz' rule)}\\

&=\int_X p(x;\theta) \dfrac{\nabla_{\theta} p(x;\theta)}{p(x;\theta)}  R(x)  \: \mathrm{d}x\\

&=\int_X p(x;\theta) \nabla_{\theta} \text{ log } p(x;\theta) R(x)  \: \mathrm{d}x \hspace{10 mm}&\text{   (Chain rule)}\\
&= \mathbb{E}_{X \sim P_{\theta}}\left[\nabla_{\theta}\text{ log } p(X;\theta)R(X)\right]\hspace{10 mm}&(2)\end{align*}   $$

The key thing to note here is that $(1)$ is not expressible as an expectation, so cannot be estimated using the sample mean from a Monte Carlo process. On the other hand, we can draw samples from our distribution $P$ to estimate $(2)$ via a Monte Carlo process.

Let's cast this in the context of a policy gradient process.

We want to find $$\underset{\theta}{\operatorname{argmax}}\hspace{1 mm}\mathbb{E}_{X \sim \pi_{\theta}}[R(X)] $$

I.e. what set of parameters leads the best performance by the policy, under some measurement function $R$. Usually, we'll measure this R using an advantage function $A$ that tells us how good some action is relative to a baseline. Note here we simplify by assuming a deterministic transition process, but the analysis is the same (and indeed transition is anyways deterministic in LLMs).

We'll do this optimization by performing stochastic gradient ascent on $\theta$ using the update rule $$\theta_{k+1}=\theta_{k} + \dfrac{\alpha}{T} \sum_{t=0}^{T-1} \nabla_{\theta_k} \text{log } \pi_{\theta_k}(a_t|s_t) \hat{A_t}   $$

where $\pi_\theta$ is the policy parameterized by $\theta$, $\hat{A_t}$ is some advantage estimate at step t, and $\alpha$ is some learning rate

The update rule takes the old parameter set, and adds the product of a learning rate and a Monte Carlo estimate of the gradient of the expectation of our advantage, as calculated in formula (2) above.

## 2. Importance Sampling

Importance sampling allows us to calculate an expectation under distribution $P$, while using data from distribution $Q$

In particular

$$\begin{align*}\mathbb{E}_{X \sim P}[f(X)] &=\int_X p(x) f(x) \: \mathrm{d}x\\

&=\int_X q(x) \dfrac{p(x)}{q(x)} f(x) \: \mathrm{d}x\\

&=\mathbb{E}_{X \sim Q}\left[\dfrac{p(X)}{q(X)}f(X)\right] 

\end{align*} $$

The usage of this in the context of policy gradient is that we can use data drawn from some policy $\pi_{\theta_\text{old}}$ to update policy $\pi_\theta$ even as the distributions drift apart.

Also note, that when our distribution policy and trained policy differ, the nice analysis on VPG above no longer applies. If we have already taken n steps, using data from some policy $\pi_{\theta_k}$ , then our update rule becomes this: $$\theta_{k+n+1}=\theta_{k+n} + \dfrac{\alpha}{T} \sum_{t=0}^{T-1} \dfrac{\pi_{\theta_{k+n}}(a_t|s_t)}{\pi_{\theta_k}(a_t|s_t)} \nabla_{\theta_k} \text{log } \pi_{\theta_k}(a_t|s_t) \hat{A_t}   $$

There's a subtlety here - the above equation only holds when the state distribution itself is invariant (or approximately invariant) over policy updates. Unfortunately there isn't really a simple way to analytically interpret this so I'm glossing over it a bit. One of the good things is that when we constrain policy updates, we will also automatically constrain the state distribution changes too.

The other reason I'm ignoring this is that I am mostly presenting this in the context of LLMs, where full deterministic trajectories are always used, so there is no state distribution issue.

## 3. Issues with taking multiple steps in VPG

That importance sampling ratio $\dfrac{\pi_{\theta_{k+n}}(a_t|s_t)}{\pi_{\theta_k}(a_t|s_t)}$ is problematic for two reasons

1. We're updating in parameter space, not policy space. This means that an update can create a new policy that is significantly different from the previous policy (creating very large or very small ratios).
2. The ratio is deceivingly simple. If we imagine language modelling, where we generate 100 tokens for example as a response for policy gradient methods, that ratio is the product of 100 different probability ratios. If our new policy attaches a 5% higher probability to each token, then the overall ratio is $1.05^{100} \sim 131$

Moreover, I mentioned that we glossed over the issue with state distribution divergence - this of course also increases the variance of this optimization process.

These factors create a lot of instability in the update process. In the PPO paper, they also ignore the state distribution issue, instead focusing on a surrogate objective that includes the importance sampling ratio. They discuss performing multiple updates without the ratio, simply approximating the ratio as =1. And they find that it leads worse results than even PPO with no clipping or KL penalty (which had terrible results, so worse than terrible). And their discussion of optimizing with the ratio but without a constraint is simply that it leads to excessively large policy updates. Hence, we need to find some effective constraint on this process.

TRPO puts a hard constraint on the difference between the new policy and the old policy using KL divergence, then solves a constrained optimization problem to find the policy that maximizes the reward within that part of the policy space (the "Trust Region" or TR in the name TRPO).

PPO instead is much closer to regular VPG. It just looks to be pessimistic about the estimated policy performance when the new potential policy significantly differs from the old policy that we sampled from - if the ratio is large, and advantage is positive, it pushes the ratio back to some fixed maximum. If the ratio is small, and advantage is negative, it pushes the ratio back to some fixed minimum. It's a practical method to remove the importance sample ratio problems. And by constraining the policy update, it seems to also practically constrain the state distribution shift.

## 4. How PPO corrects this

Let's take a slightly deeper look at what this means exactly.

We recast the importance sampling ratio as $ r_t(\theta)=\dfrac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}$ which is exactly what appears in the PPO paper.

Then, the regular VPG objective is $$\hat{\mathbb{E}}_t[r_t(\theta)\hat{A_t}]$$

where $\hat{A_t}$ is an estimator of the advantage, $\hat{\mathbb{E}}_t$ represents an empirical average over the old distribution, 

When we do only one update on data from a policy, then resample from the new policy, we have $r_t(\theta) =1  $ , and so we get our original analysis from section 1. When we do multiple updates, this just represents importance sampling and is the natural objective to maximize expectation under $\pi_\theta$ while drawing data from $\pi_{\theta_\text{old}}$ 

PPO adapts this to use the following objective $$\hat{\mathbb{E}}_t[\min(r_t(\theta)\hat{A_t},\text{clip}(r_t(\theta),1- \epsilon,1+ \epsilon)\hat{A_t})]$$

where $\epsilon$ is a hyperparameter, suggested as $0.2$.

The left part of the minimum is our original objective, the right part clips the ratio into an interval so that we get a pessimistic version of the original objective. This prevents excessively large or small ratios (i.e. where the distributions differ greatly) in situations that could cause excessively optimistic updates in policy.  

Here are some concrete examples of how this clipping changes our update vs the standard objective, calling $L$ our clipped objective and $M$ our standard VPG objective

$r_t(\theta) = 100$ , $\hat{A_t}=5$, leads to $L=\min(100*5,1.2*5)=6, M=500$ (when advantage positive and ratio large, ratio gets clipped down)

$r_t(\theta) = 100$ , $\hat{A_t}=-5$, leads to $L=\min(100*-5,1.2*-5)=-500, M=-500$ (when advantage negative and ratio large, we use full negative objective)

$r_t(\theta) = 0.1$ , $\hat{A_t}=100$, leads to $L=\min(0.1*100,0.8*100)=10,M=10$ (when advantage positive and ratio small, we use the small ratio)

$r_t(\theta) = 0.1$ , $\hat{A_t}=-100$, leads to $L=\min(0.1*-100,0.8*-100)=-80,M=-10$ (when advantage negative and ratio small, we use the clipped ratio which is negative and larger in magnitude)

You can see that the effect of this is that the objective $L$ is always pessimistic relative to original VPG objective $M$.

The overall takeaway from this should be that while PPO was originally developed as a practical alternative to TRPO, a complex constrained optimization method, there's a simple alternative interpretation as seeing it as controlling the importance sampling ratio that emerges when we take multiple steps in vanilla policy gradient.

